In [1]:
library(tidyverse)
library(data.table)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.1  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [28]:
liftOver_df <- fread(
    'ukb24983_cal_cALL_v2_1.hg38.bed', sep='\t'
)
colnames(liftOver_df) <- c('CHROM', 'POS_S', 'POS_E', 'BED_ID')

In [29]:
bim37_df <- fread(
    '/oak/stanford/groups/mrivas/private_data/ukbb/24983/cal/pgen/ukb24983_cal_cALL_v2_1.bim',
    sep='\t'   
)
colnames(bim37_df)<-c('CHROM', 'ID', 'CM', 'hg37_POS', 'hg37_A1', 'hg37_A2')

In [30]:
bim38_df <- bim37_df %>% 
mutate(
    BED_ID = paste0(ID, ':', 'chr', CHROM, ':', hg37_POS, ':', hg37_A1, ':', hg37_A2)
) %>%
left_join(
    liftOver_df %>% 
    mutate(hg38_POS = POS_S + 1) %>% 
    select(BED_ID, hg38_POS),
    by='BED_ID'
) %>%
select(
    CHROM, ID, CM, hg38_POS, hg37_A1, hg37_A2
)

In [31]:
bim38_df %>% head()

CHROM,ID,CM,hg38_POS,hg37_A1,hg37_A2
1,rs28659788,0,787927,G,C
1,rs116587930,0,792461,A,G
1,rs116720794,0,794252,T,C
1,rs3131972,0,817341,G,A
1,rs12184325,0,818725,T,C
1,rs3131962,0,821224,G,A


In [32]:
bim38_df %>% filter(
    is.na(hg38_POS)
) %>% dim()

[1] 293   6

In [33]:
bim38_df %>% filter(
    is.na(hg38_POS)
) %>% head()

CHROM,ID,CM,hg38_POS,hg37_A1,hg37_A2
1,rs4648756,0,NA,C,G
1,rs3936009,0,NA,T,G
1,rs6541019,0,NA,G,A
1,rs7543599,0,NA,T,C
1,rs6541027,0,NA,T,C
1,rs3010948,0,NA,T,C


In [34]:
bim38_df %>% filter(
    is.na(hg38_POS)
) %>% 
replace_na(list(hg38_POS = 0)) %>% head()

CHROM,ID,CM,hg38_POS,hg37_A1,hg37_A2
1,rs4648756,0,0,C,G
1,rs3936009,0,0,T,G
1,rs6541019,0,0,G,A
1,rs7543599,0,0,T,C
1,rs6541027,0,0,T,C
1,rs3010948,0,0,T,C


In [36]:
bim38_df %>% 
replace_na(list(hg38_POS = 0)) %>% 
fwrite(
    'ukb24983_cal_cALL_v2_1.hg38.bim', sep='\t', row.names=F, col.names=F
)